In [15]:
#imports.... Run this each time after restarting the Kernel
#!pip install -e git+https://github.com/ibm-watson-data-lab/pixiedust@david-gateway-branch#egg=pixiedust
#!pip install --user --upgrade pixiedust
#!pip install watson_developer_cloud
import pixiedust
import json
import pandas
import datetime
import watson_developer_cloud as watson
import json
import time
import re
import pyspark
import sys
import pandas as pd
from botocore.client import Config
import ibm_boto3




In [16]:
# The code was removed by DSX for sharing.

In [17]:
# The code was removed by DSX for sharing.

In [18]:

client = ibm_boto3.client(service_name='s3', 
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')




In [33]:
#NLC Part 1

from watson_developer_cloud import NaturalLanguageClassifierV1


natural_language_classifier = NaturalLanguageClassifierV1(
    username=username_NLC,
    password=password_NLC)

chunk_size = 30

def chunk_transcript(transcript, chunk_size):
    transcript = transcript.split(' ')
    return [ transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size) ] # chunking data
    

def process_text(text):
    transcript=''
    for sentence in json.loads(text)['results']:
        transcript = transcript + sentence['alternatives'][0]['transcript'] # concatenate sentences
    transcript = chunk_transcript(transcript, chunk_size) # chunk the transcript
    return transcript

def classify(file_name):
    streaming_body = client.get_object(Bucket = credentials['BUCKET'], Key = file_name)['Body']
    transcript=streaming_body.read().decode("utf-8")
    analysis = {}
    for chunk in process_text(transcript):
        chunk = ' '.join(chunk)
        print(chunk)
        analysis[chunk] = natural_language_classifier.classify(classifier_id, chunk)
    client.put_object(Bucket = credentials['BUCKET'], Key = file_name[0].split('_')[0]+'_nlc', Body= json.dumps(analysis))
    return analysis


def classify_transcript(file_name):
    status = natural_language_classifier.get_classifier(classifier_id)
    print (json.dumps(status, indent=2))
    if status['status'] == 'Available':
        classes = classify(file_name)
        print(json.dumps(classes, indent=2))


In [34]:
text_files = ['sample1-addresschange-positive_text.json', 'sample2-address-negative_text.json', 'sample3-shirt-return-weather-chitchat_text.json', 'sample4-angryblender-sportschitchat-recovery_text.json', 'sample5-calibration-toneandcontext_text.json'] # we add audio files to COS pre-conference


classify_transcript(text_files[0])


{
  "created": "2016-08-17T08:01:31.570Z",
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/340008x87-nlc-967",
  "classifier_id": "340008x87-nlc-967",
  "name": "sortinghat3",
  "language": "en",
  "status": "Available",
  "status_description": "The classifier instance is now available and is ready to take classifier requests."
}
good morning can you give me some help I'd like to change my address please my name is Ryan Smith I am from Sacramento California that's right my phone number
is five five five one two one two yes that's me my old address is number one two three oak street my new address is five six seven pine street
yes and the zip is nine zero two one zero yep that's right now the phone number stays the same that's right I would like to keep all the options
of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it
thanks have a good day bye bye 
{
 